# Tutorial CV1 - Photon redirection

This tutorial runs PennyLane's "hello world" example of continuous-variable quantum computation with the StrawberryFields plugin. Starting with a photon in Mode 1, the goal is to optimize a beam splitter to redirect the photon to Mode 2. 

### Imports

First we need to import openqml, as well as openqml's version of numpy. This allows us to automatically compute gradients for functions that manipulate numpy arrays, including quantum functions.

In [2]:
import openqml as qm
from openqml import numpy as np
from openqml.optimize import GradientDescentOptimizer, MomentumOptimizer

Next, create a "device" to run the quantum node. We use StrawberryFields to simulate a photonic quantum processor with two quantum modes.

In [3]:
dev = qm.device('strawberryfields.fock', wires=2, cutoff_dim=10)

ImportError: module 'openqml_sf' has no attribute 'StrawberryFieldsFock'

### Quantum function

We define a quantum function which "puts" one photon into the first mode and sends both modes through a beam splitter. 

In [3]:
@qm.qfunc(dev)
def circuit(weights):

    qm.FockState(1, [0])
    qm.Beamsplitter(weights[0], weights[1], [0, 1])

    return qm.expectation.Fock(1)

This function uses openqml to run the following quantum circuit:

<img src="figures/redirection_circuit.png">

Depending on the transmission parameter $w_0$, the photon remains in the first mode (i.e., $w_1 = n \pi$ with integer $n$), gets redirected to the second mode (i.e., $w_1 = n\pi / 2$) or is in a superposition of being in both modes. The phase parameter $w_2$ does not have any influence on the result, so we expect it to have a zero gradient and not to get updated at all.

### Objective

Next, we define a cost that quantifies whether the redirection is successful. Since we want to maximize the expectation of the Fock measurement in the second mode, we minize the negative of the circuit output.

In [4]:
def objective(weights):
    return -circuit(weights)

This objective has the following optimization landscape: 
 
 <img src="figures/redirection_landscape.png" width="450">

As stated above, the gradient does not depend on the phase parameter $w_2$. 

### Optimization

The initial values of the transmission and phase parameters are set to near-zero. This corresponds to an identity gate, in other words, the circuit leaves the photon in the first mode. *Note that at zero exactly the gradient is zero and the optimization algorithm will not descent from the maximum.*

In [5]:
weights0 = np.array([0.01, 0.01])
print('Initial rotation angles:', weights0)

Initial rotation angles: [0.01 0.01]


We choose a simple Gradient Descent Optimizer and update the weights for 10 steps.

In [6]:
o = GradientDescentOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(objective, weights)
    if step%5==0:
        print('Objective after step {:5d}: {:.7f}'.format(step, objective(weights)) )

print()
print('Optimized rotation angles:', weights)

Objective after step     5: -0.0001000
Objective after step    10: -0.0001000
Objective after step    15: -0.0001000
Objective after step    20: -0.0001000
Objective after step    25: -0.0001000
Objective after step    30: -0.0001000
Objective after step    35: -0.0001000
Objective after step    40: -0.0001000
Objective after step    45: -0.0001000
Objective after step    50: -0.0001000
Objective after step    55: -0.0001000
Objective after step    60: -0.0001000
Objective after step    65: -0.0001000
Objective after step    70: -0.0001000
Objective after step    75: -0.0001000
Objective after step    80: -0.0001000
Objective after step    85: -0.0001000
Objective after step    90: -0.0001000
Objective after step    95: -0.0001000
Objective after step   100: -0.0001000

Optimized rotation angles: [0.01 0.01]


Starting at a different offset, we train the MomentumOptimizer, which improves on gradient descent by making an update dependent on the previous gradient.

In [7]:
weights0 = np.array([-0.01, 0.01])
print('Initial rotation angles:', weights0)

o = MomentumOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(objective, weights)
    if step%5==0:
        print('Objective after step {:5d}: {:.7f}'.format(step, objective(weights)) )

print()
print('Optimized rotation angles:', weights)

Initial rotation angles: [-0.01  0.01]
Objective after step     5: -0.0001000
Objective after step    10: -0.0001000
Objective after step    15: -0.0001000
Objective after step    20: -0.0001000
Objective after step    25: -0.0001000
Objective after step    30: -0.0001000
Objective after step    35: -0.0001000
Objective after step    40: -0.0001000
Objective after step    45: -0.0001000
Objective after step    50: -0.0001000
Objective after step    55: -0.0001000
Objective after step    60: -0.0001000
Objective after step    65: -0.0001000
Objective after step    70: -0.0001000
Objective after step    75: -0.0001000
Objective after step    80: -0.0001000
Objective after step    85: -0.0001000
Objective after step    90: -0.0001000
Objective after step    95: -0.0001000
Objective after step   100: -0.0001000

Optimized rotation angles: [-0.01  0.01]


(PLOT OPTIMIZER STEPS)

In [ ]:
# predict a range of values between -1 and 1
x_axis = np.linspace(-1, 1, 50)
predictions = [quantum_neural_net(weights, x=x) for x in x_axis]

In [ ]:
plt.figure()
plt.plot(x_axis, predictions, color='#3f9b0b', marker='o', zorder=1)
plt.scatter(X, Y, color='#fb2943', marker='o', zorder=2, s=75)
plt.xlabel('Input', fontsize=18)
plt.ylabel('Output', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.tick_params(axis='both', which='minor', labelsize=16)
plt.show()